# Cooking a simple neural network library

## Ingredients

- `numpy`
- [a loss function](#Loss-function)
- [some layers](#Layers)
- [a neural net](#Neural-network)
- [an optimizer](#Optimizer)
- [a batch data provider](#Batch-generator)
- [a training routine](#Training)
- \+ an example

Hopefully by the end of this tutorial you will have an understanding of the building blocks needed for training (deep) neural networks. 

## Foreword

We will purely rely on numpy for this tutorial. Make sure to import it here.

In [1]:
import numpy as np

#### Object Oriented Python

Object-oriented Python, a.k.a _classes_, will be used intensively in this tutorial.  
For those not familiar with Python classes, know that you will only be required to write some definitions and Python code **within the** class **methods** and **not** actually **write any class**.  

If you want to know more about Python classes, here is a step by step [tutorial](https://aboucaud.github.io/slides/2016/python-classes).

#### Type hints

This notebook uses a feature from Python 3 called ***type hints*** or ***type annotations*** (see [PEP 0526](https://www.python.org/dev/peps/pep-0526/)). This acts like optional static typing since Python will still run if the type does not match, but has two main advantages IMO:
- make sure you understand what you're doing
- act like documentation for an external user

The types for the base Python objects (lists, dicts, iterables) can be found in the [`typing` library](https://docs.python.org/3/library/typing.html).
For instance, here are all the needed imports for this tutorial.

In [2]:
from typing import (Dict, Tuple, Callable, 
                    Sequence, Iterator, NamedTuple)

Any other Python object can serve as a type. We will use the `numpy.ndarray` in this tutorial to mock a tensor. We thus create a `Tensor` object to use as type hint throughout the code, and an object `Func` for a function that acts element-wise on a tensor and returns a tensor.

In [3]:
from numpy import ndarray as Tensor

Func = Callable[[Tensor], Tensor]

#### Checking the type

If type hints are optional for Python, they can still be used to actually check the consistency of the code. For this task, there is a module called [`mypy`](https://github.com/python/mypy) that can be used (not in this tutorial). 

Check out the [doc](http://mypy-lang.org/) if you are interested.

---

## Loss function

A loss function measures how good our predictions are,
we can use this to adjust the parameters of our network

Here is generic loss class. It implements the computation of **the loss** from the true label and the predicted one, as well as **the gradient of the loss** for the backpropagation.

In [4]:
class Loss:
    def loss(self, predicted: Tensor, actual: Tensor) -> float:
        raise NotImplementedError

    def grad(self, predicted: Tensor, actual: Tensor) -> Tensor:
        raise NotImplementedError

### Exercice #1 - mean square error loss

***3 min*** - *Implement the `MeanSquareError` class*

For info, the loss function is

$$MSE(y_{true}, y_{pred}) = \sum \left(y_{pred} - y_{true}\right) ^ 2$$



In [5]:
class MeanSquareError(Loss):
    def loss(self, predicted: Tensor, actual: Tensor) -> float:
        return np.sum((predicted - actual) ** 2)
    
    def grad(self, predicted: Tensor, actual: Tensor) -> Tensor:
        return 2 * (predicted - actual)

## Layers

Our neural nets will be made up of layers.
Each layer needs to pass its inputs forward
and propagate gradients backward. For example,
a neural net might look like
inputs -> Linear -> Tanh -> Linear -> output

In [6]:
class Layer:
    def __init__(self) -> None:
        self.params: Dict[str, Tensor] = {}
        self.grads: Dict[str, Tensor] = {}

    def forward(self, inputs: Tensor) -> Tensor:
        raise NotImplementedError

    def backward(self, grad: Tensor) -> Tensor:
        raise NotImplementedError

### Exercice #2 - linear layer

***5 min*** - *Implement the `forward` and `backward` methods of the linear layer.*

In [20]:
class Linear(Layer):
    """
    Inputs are of size (batch_size, input_size)
    Outputs are of size (batch_size, output_size)
    """
    def __init__(self, input_size: int, output_size: int) -> None:
        # Inherit from base class Layer
        super().__init__()
        # Initialize the weights and bias with random values
        self.params["w"] = np.random.randn(input_size, output_size)
        self.params["b"] = np.random.randn(output_size)

    def forward(self, inputs: Tensor) -> Tensor:
        """
        inputs shape is (batch_size, input_size)
        """
        self.inputs = inputs
        # Compute here the feed forward pass
        # or self.params['w'].T @ inputs.T + + self.params['b']
        return inputs @ self.params['w'] + self.params['b'] 
        

    def backward(self, grad: Tensor) -> Tensor:
        """
        grad shape is (batch_size, output_size)
        """
        # Compute here the gradient parameters for the layer
        self.grads["w"] = grad.T @ self.inputs
        self.grads["b"] = np.sum(grad , axis=0)
        # Compute here the feed backward pass
        return grad @ self.params['w'].T    

### Activation layers

In [8]:
class Activation(Layer):
    """
    An activation layer just applies a function
    elementwise to its inputs
    """
    def __init__(self, f: Func, f_prime: Func) -> None:
        super().__init__()
        self.f = f
        self.f_prime = f_prime

    def forward(self, inputs: Tensor) -> Tensor:
        self.inputs = inputs
        return self.f(inputs)

    def backward(self, grad: Tensor) -> Tensor:
        """
        if y = f(x) and x = g(z)
        then dy/dz = f'(x) * g'(z)
        """
        return self.f_prime(self.inputs) * grad

### Exercice #3 - tanh and sigmoid

***5 min*** - *Implement the hyperbolic tangent and sigmoid layers and their derivatives.*


In [9]:
def tanh(x: Tensor) -> Tensor:
    # Write here the tanh function
    return np.tanh(x) 

def tanh_prime(x: Tensor) -> Tensor:
    # Write here the derivative of the tanh
    y = tanh(x)
    return 1 - y ** 2

class Tanh(Activation):
    def __init__(self):
        super().__init__(tanh, tanh_prime)
        

def sigmoid(x: Tensor) -> Tensor:
    # Write here the sigmoid function
    return 1 / (1 + np.exp(-x))

def sigmoid_prime(x: Tensor) -> Tensor:
    # Write here the derivative of the sigmoid
    y = sigmoid(x)
    return y * (1 - y)

class Sigmoid(Activation):
    def __init__(self):
        super().__init__(sigmoid, sigmoid_prime)

## Neural network

A neural net is a collection of layers and should behave closely with a `forward` and a `backward` pass.

In addition, we add a method `params_and_grads` that will be used by the optimizer to update the values of the weights and bias of each layer.

In [10]:
class NeuralNet:
    def __init__(self, layers: Sequence[Layer]) -> None:
        self.layers = layers

    def forward(self, inputs: Tensor) -> Tensor:
        """
        The forward pass takes the layers in order
        """
        for layer in self.layers:
            inputs = layer.forward(inputs)
        return inputs

    def backward(self, grad: Tensor) -> Tensor:
        """
        The backward pass is the other way around
        """
        for layer in reversed(self.layers):
            grad = layer.backward(grad)
        return grad

    def params_and_grads(self) -> Iterator[Tuple[Tensor, Tensor]]:
        for layer in self.layers:
            for name, param in layer.params.items():
                grad = layer.grads[name]
                yield param, grad

## Optimizer

The role of the optimizer is to adjust the network parameters based on the gradients computed during backpropagation.

The main attribute of an optimizer is the _learning rate_ (a.k.a. `lr`), which defines the size of the jump taken in the direction of the gradients. 

In [11]:
class Optimizer:
    def step(self, net: NeuralNet) -> None:
        raise NotImplementedError

### Exercice #4 - Stochastic Gradient Descent

***2 min*** - write the optimizer step

Here we have a very basic implementation of a _Stochastic Gradient Descent_ (a.k.a. `SGD`). 

The step that needs to be written iterates over the neural network layers and updates the layers parameters in the direction _opposite_ to the gradient.

In [12]:
class SGD(Optimizer):
    def __init__(self, lr: float = 0.01) -> None:
        self.lr = lr

    def step(self, net: NeuralNet) -> None:
        for param, grad in net.params_and_grads(): 
            # Write here the parameters update
            param -= self.lr * grad

## Batch generator

It can be costly to compute the gradients and update the weights after every entry of the training dataset. In order to minimize such computational cost, the inputs of the network are traditionally fed in batches and the gradients are thus averages over those batches of data.

A batch size of 32 is a default in multiple training sets. Some recent [study](https://arxiv.org/abs/1804.07612) claims this number is the perfect balance between computing efficiency and training stability.

During an epoch the network will iterate over the whole dataset. Adding some shuffling in the process ensures the batches are not fed exactly in the same order at each epoch.

In [13]:
Batch = NamedTuple("Batch", [("inputs", Tensor), ("targets", Tensor)])


class DataIterator:
    def __call__(self, inputs: Tensor, targets: Tensor) -> Iterator[Batch]:
        raise NotImplementedError

        
class BatchIterator(DataIterator):
    def __init__(self, batch_size: int = 32, shuffle: bool = True) -> None:
        self.batch_size = batch_size
        self.shuffle = shuffle

    def __call__(self, inputs: Tensor, targets: Tensor) -> Iterator[Batch]:
        starts = np.arange(0, len(inputs), self.batch_size)
        if self.shuffle:
            np.random.shuffle(starts)

        for start in starts:
            end = start + self.batch_size
            batch_inputs = inputs[start:end]
            batch_targets = targets[start:end]
            yield Batch(batch_inputs, batch_targets)

## Training

The training routine uses all objects defined above and executes actions **in the right order** to train the neural network.

The dataset being usually small with respect to the number of free parameters of the neural net, going through the dataset multiple times during the training is a necessity. This corresponds to the number of epochs, which has to be specified.

### Exercise #5 - build the training routine

***5 min*** - write the sequential steps needed for training at each epoch

_Hints_:
- feed forward
- compute the loss and the gradients
- feed backwards
- update the net

In [14]:
def train(net: NeuralNet, inputs: Tensor, targets: Tensor,
          loss: Loss = MeanSquareError(), 
          optimizer: Optimizer = SGD(),
          iterator: DataIterator = BatchIterator(),
          num_epochs: int = 5000) -> None:
    for epoch in range(num_epochs):
        epoch_loss = 0.0
        for batch in iterator(inputs, targets):
            # Write here the various steps (in order) needed 
            # at each epoch
            batch_pred = net.forward(batch.inputs)
            epoch_loss += loss.loss(batch_pred, batch.targets)
            grads = loss.grad(batch_pred, batch.targets)
            net.backward(grads)
            optimizer.step(net)
        # Print status every 50 iterations
        if epoch % 50 == 0:
            print(epoch, epoch_loss)

## Application exercise

Now that you have build your own neural network library, let's use it to solve a problem and then put it in application.

### XOR

Canonical problem in ML as there is not linear way to map the inputs to the output.

```
[0, 0] => 0  
[0, 1] => 1  
[1, 0] => 1  
[1, 1] => 0  
```

In [15]:
X = np.array([[0, 0], [1, 0], [0, 1], [1, 1]])
y = np.array([[0], [1], [1], [0]])

def print_xor_results(inputs: Tensor, targets: Tensor, predictions: Tensor) -> None:
    print('\nX => y => y_pred => round(y_pred)')
    for x, y, z in zip(inputs, targets, predictions):
        print(f'{x} => {y} => {z} => {z.round()}')
        
def train_xor(net: Optimizer, inputs: Tensor, targets: Tensor, epochs: int = 2000):
    train(net, inputs, targets, num_epochs=epochs)
    y_pred = net1.forward(inputs)
    print_xor_results(inputs, targets, y_pred)

Here is an attempt at solving the XOR problem using a single linear layer

In [16]:
net1 = NeuralNet([
    Linear(input_size=2, output_size=1),
])

train_xor(net1, X, y)

0 3.0689150503291245
50 1.0709828521313474
100 1.0172483073449088
150 1.004502346303835
200 1.0012232369875043
250 1.0003392315573345
300 1.000095027673406
350 1.0000267483149394
400 1.0000075463015208
450 1.0000021312732843
500 1.000000602231979
550 1.0000001702125785
600 1.0000000481136062
650 1.0000000136008762
700 1.0000000038448247
750 1.000000001086904
800 1.0000000003072615
850 1.0000000000868612
900 1.0000000000245552
950 1.0000000000069418
1000 1.0000000000019624
1050 1.0000000000005547
1100 1.0000000000001568
1150 1.0000000000000442
1200 1.0000000000000124
1250 1.0000000000000036
1300 1.0000000000000009
1350 1.0000000000000002
1400 1.0000000000000002
1450 1.0
1500 1.0
1550 1.0
1600 1.0
1650 1.0
1700 0.9999999999999999
1750 1.0
1800 1.0
1850 1.0
1900 1.0
1950 1.0

X => y => y_pred => round(y_pred)
[0 0] => [0] => [0.5] => [1.]
[1 0] => [1] => [0.5] => [1.]
[0 1] => [1] => [0.5] => [1.]
[1 1] => [0] => [0.5] => [0.]


This does not work, as expected since XOR is a typical non-linear problem

### Exercise #6 - solve XOR with a Neural Net

Write a more advanced neural net (using additional linear and activation layers) until the predictions match the target values.

In [17]:
# Write a deeper neural net and see the results
net2 = NeuralNet([
    # Add the layers here
    Linear(input_size=2, output_size=2),
    Tanh(),
    Linear(input_size=2, output_size=1)
])

train_xor(net2, X, y)

0 2.0464083049246473
50 0.5588488755947275
100 0.3530527970198266
150 0.21608251310765006
200 0.12777290955629686
250 0.07339733585567376
300 0.04097060211774631
350 0.02225966133252475
400 0.011813691461479658
450 0.0061516802240413546
500 0.0031564491807326706
550 0.0016017340690579855
600 0.0008061925416301813
650 0.00040338168360640945
700 0.00020097707728772884
750 9.982952789630974e-05
800 4.948068611946886e-05
850 2.448782034535731e-05
900 1.2105895911868244e-05
950 5.980178055187882e-06
1000 2.952563206168591e-06
1050 1.4572061873788667e-06
1100 7.189985206722174e-07
1150 3.5469436502879204e-07
1200 1.7495400026026783e-07
1250 8.628864083025046e-08
1300 4.25554705401538e-08
1350 2.098637898290195e-08
1400 1.0349177176883692e-08
1450 5.10345675105918e-09
1500 2.5166119093083055e-09
1550 1.2409756940045327e-09
1600 6.119373282828312e-10
1650 3.017506811574992e-10
1700 1.4879485571829762e-10
1750 7.337133337642976e-11
1800 3.6179627418341434e-11
1850 1.784026065611403e-11
1900 8.7

### Exercise #7 - write the same model using Keras

#### Keras installation

In a terminal enter the following command to install Keras
```
conda install --name adaix keras
```

#### Task
Based on the examples given in the [neural network part](https://aboucaud.github.io/adaix-ml-tutorial/slides/hands-on-deep-learning/#39) of the lecture, as well as the section on loss and optimizers, write the exact same example using the Keras library.




In [19]:
# Write down the keras model below
#---------------------------------
# Star by the necessary imports

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Activation
from keras.optimizers import SGD
from keras.metrics import mean_squared_error

# Write the model architecture

model = Sequential()

model.add(Dense(2, input_dim=2))
model.add(Activation('tanh'))
model.add(Dense(1))

# Compile the model

opt = SGD(lr=0.05)

model.compile(optimizer=opt, loss=mean_squared_error)

# Train the model (no validation_split required here)

model.fit(X, y, epochs=3000, verbose=0)

#---------------------------------

# Once trained, this will then predict the values (equivalent of `.forward()`)
y_pred_keras = model.predict(X)

# And print the results
print_xor_results(X, y, y_pred_keras)


X => y => y_pred => round(y_pred)
[0 0] => [0] => [2.1457672e-06] => [0.]
[1 0] => [1] => [0.9999971] => [1.]
[0 1] => [1] => [0.99999684] => [1.]
[1 1] => [0] => [1.5497208e-06] => [0.]


### Star-galaxy separation

Use the neural net library you've just created to perform the star-galaxy classification you worked on this morning (cf. [notebook](star-galaxy_classification.ipynb)) using Neural Networks !

To help you, here is a helper function that will reload the data and perform the preprocessing for you (you're welcome :-)

In [ ]:
%load ../code/load_star_gal_data.py

In [ ]:
X, y = get_star_gal_data()

## Acknowledgements

The idea and the code for this tutorial have been for the most part inspired by the video "Deep Learning Madness" https://youtu.be/o64FV-ez6Gw by [Joel Grus](https://twitter.com/joelgrus)